**This notebook reads the json files containing the playlists and convert them into csv files to be updated to a PostgreSQL database**

In [1]:
import json
import os
import pandas as pd
from IPython.display import clear_output

In [2]:
# Convert the nested dictionaries in the initial json file to a flat row that is concatenated to the dataframe
def process_json_data(playlists):
  global playlist_tracks_df

  tracks_dic_df = pd.json_normalize(playlists, record_path=['tracks'], meta=['pid','name', 'num_tracks'])

  playlist_tracks_df = pd.concat([playlist_tracks_df, tracks_dic_df], ignore_index=True)


In [3]:
# Reads a group of json files for data extraction
def process_mpd(slices,kk): 
    count_slice=kk
    count_playlist=0
    for filename in sorted(slices):

        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join(('../data/', filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice["playlists"]:
                print('slice='+str(count_slice)+';  playlist='+str(count_playlist))
                process_json_data(playlist)
                clear_output(wait=True)
                count_playlist +=1
        count_slice +=1

In [4]:
# To define the location of the JSON files
path_to_slices='../data/'
filenames = os.listdir(path_to_slices)


In [ ]:

# To define the beginning of the slices to be read
k=0
k_end=1000
k_inc=10    # 10 is the recommended number of playlists to be concatenated into a single dataframe and saved as an individual csv file.
            # Anything longer will see a dramatic slow down caused by the concatenation process as the dataframe gets bigger.

for kk in range(k,k_end,k_inc):
    # To initialize the global dataframe
    playlist_tracks_df=pd.DataFrame()
    process_mpd(filenames[kk:kk+k_inc],kk)

    # To reorder the columns
    playlist_tracks_df=playlist_tracks_df.reindex(columns=['pid','name','num_tracks','pos','track_name','track_uri','artist_name','artist_uri','album_name','album_uri', 'duration_ms'])
    # To update the column names
    playlist_tracks_df=playlist_tracks_df.rename(columns={'pid':'playlist_id','name':'playlist_name','pos':'track_pos'})

    # To save the dataframe as a CSV file. 
    file_name='data/playlist_tracks_'+str(kk)+'_'+str(kk+k_inc-1)+'.csv'
    playlist_tracks_df.to_csv(file_name,index=False)

---------------------------------------
*** Testing section ***

In [2]:
# To visualize a single playlist
playlist_tracks_df=pd.read_csv('enter_path_to_file')
# playlist_tracks_df=pd.read_csv('../spotify_million_playlist_dataset/src/data_playlists/playlist_tracks_0_19.csv')
pid=0       # playlist number present in the csv file 
playlist_tracks_df[playlist_tracks_df['playlist_id']==pid]

,playlist_id,playlist_name,num_tracks,track_pos,track_name,track_uri,artist_name,artist_uri,album_name,album_uri,duration_ms
0,0,Throwbacks,52,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863
1,0,Throwbacks,52,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800
2,0,Throwbacks,52,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933
3,0,Throwbacks,52,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266
4,0,Throwbacks,52,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600
5,0,Throwbacks,52,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,250373
6,0,Throwbacks,52,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,223440
7,0,Throwbacks,52,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,225560
8,0,Throwbacks,52,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,271333
9,0,Throwbacks,52,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,235213


In [3]:
playlist_tracks_df.shape

(1339962, 11)

To check the length of the longest string in the column (for VARCHAR())

In [41]:
import numpy as np

In [5]:
playlist_tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339962 entries, 0 to 1339961
Data columns (total 11 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   playlist_id    1339962 non-null  int64 
 1   playlist_name  1339962 non-null  object
 2   num_tracks     1339962 non-null  int64 
 3   track_pos      1339962 non-null  int64 
 4   track_name     1339955 non-null  object
 5   track_uri      1339962 non-null  object
 6   artist_name    1339960 non-null  object
 7   artist_uri     1339962 non-null  object
 8   album_name     1339962 non-null  object
 9   album_uri      1339962 non-null  object
 10  duration_ms    1339962 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 112.5+ MB


list of column names with text :['playlist_name', 'track_name', 'track_uri', 'artist_name', 'artist_uri', 'album_name', 'album_uri']

In [68]:
# finds the longest string(s) in a column
lengths = playlist_tracks_df["album_uri"].str.len()
argmax = np.where(lengths == lengths.max())[0]
playlist_tracks_df.iloc[argmax]['album_uri']

0          spotify:album:6vV5UrXcfyQD1wu4Qo2I9K
1          spotify:album:0z7pVBGOD7HCIB7S8eLkLI
2          spotify:album:25hVFAxTlDvXbx2X2QkUkE
3          spotify:album:6QPkyl04rXwTGlGlcYaRoW
4          spotify:album:6NmFmPX56pcLBOFMhIiKvF
                           ...                 
7322474    spotify:album:0o2oPAxKGui4tvrrNgDtkc
7322475    spotify:album:40V7QExJNlpQZaxZf1JTF5
7322476    spotify:album:6YXkbmsVABrzPBO6Vknj2P
7322477    spotify:album:2ObOh59P7oEmLSvqSUjXxc
7322478    spotify:album:2LYwooMTH1iJeBvWyXXWUf
Name: album_uri, Length: 7322479, dtype: object

In [69]:
# check the length of a string in a given playlist
len(playlist_tracks_df.iloc[7322474]['album_uri'])

36

max length:  
name: 75  
track_name: 253  
track_uri: 32  
artist_name: 267  
artist_uri: 37  
album_name: 286  
album_uri: 36


****************************************